In [2]:
from IPython.display import Audio
Audio('example.wav')

In [3]:
import whisper
tiny_model = whisper.load_model('tiny')

In [4]:
tiny_model.transcribe('example.wav')

C:\Users\Zach\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' She even seemed mildly amused at the attention she attracted.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.44,
   'text': ' She even seemed mildly amused at the attention she attracted.',
   'tokens': [50364,
    1240,
    754,
    6576,
    15154,
    356,
    669,
    4717,
    412,
    264,
    3202,
    750,
    15912,
    13,
    50536],
   'temperature': 0.0,
   'avg_logprob': -0.22374729812145233,
   'compression_ratio': 1.0,
   'no_speech_prob': 0.004775787238031626}],
 'language': 'en'}

In [6]:
%%timeit
result=tiny_model.transcribe('example.wav', fp16=False)

1.2 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
small_model=whisper.load_model('small')
small_model.transcribe('example.wav', fp16=False)

{'text': ' She even seemed mildly amused at the attention she attracted.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.44,
   'text': ' She even seemed mildly amused at the attention she attracted.',
   'tokens': [50364,
    1240,
    754,
    6576,
    15154,
    356,
    669,
    4717,
    412,
    264,
    3202,
    750,
    15912,
    13,
    50536],
   'temperature': 0.0,
   'avg_logprob': -0.17723101377487183,
   'compression_ratio': 1.0,
   'no_speech_prob': 0.0031393084209412336}],
 'language': 'en'}

In [ ]:
%%timeit
result=small_model.transcribe('example.wav', fp16=False)

In [ ]:
with open('credentials', 'r') as f:
    secret_key = f.read()
    
import openai
openai.api_key = secret_key

In [ ]:
audio_file = open("example.wav", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
transcript

In [ ]:
import pandas as pd
rnz_metadata = pd.read_csv('data/morning_report.csv')
rnz_metadata.head()

In [ ]:
rnz_metadata.loc[1,'Text']

In [ ]:
import wget
import os
rnz_data_dir = 'data/rnz'
rnz_raw_dir = os.path.join(rnz_data_dir, 'raw')
os.makedirs(rnz_raw_dir, exist_ok=True)
filepath=wget.download(rnz_metadata.loc[1,'URL'],out=rnz_raw_dir)
Audio(filepath)

In [ ]:
# helper function
def timestamp_to_ms(timestamp):
    m, s = timestamp.split(':')
    s, ss = s.split('.')
    m, s, ss = [int(t) for t in [m,s,ss]]
    return m*60*1000+s*1000+ss

from pydub import AudioSegment
import re

clip_filepath = re.sub('/raw/','/clips/',filepath)
rnz_clips_dir = os.path.join(rnz_data_dir, 'clips')
os.makedirs(rnz_clips_dir, exist_ok=True)

start_ms = timestamp_to_ms(rnz_metadata.loc[1,'Start'])
end_ms = timestamp_to_ms(rnz_metadata.loc[1,'End'])

sound = AudioSegment.from_mp3(filepath)
sound[start_ms:end_ms]

In [ ]:
sound[start_ms:end_ms].export(clip_filepath, format="mp3")

tiny_model.transcribe(clip_filepath, fp16=False)['text']

In [ ]:
def transcribe_api(mp3_path):
    with open(mp3_path, "rb") as audiofile:
        transcript=openai.Audio.transcribe("whisper-1", audiofile, response_format='text', language='en')
    return transcript.strip()

In [ ]:
transcribe_api(clip_filepath)